In [1]:
import pandas as pd

In [2]:
airbnb = pd.read_csv('cleaned.csv')

In [3]:
airbnb.head()

,name,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,host
0,COZICOMFORT LONG TERM STAY ROOM 2,North Region,Woodlands,Private room,83,180,1,2013-10-21,0.01,2,365,266763: Francesca
1,Pleasant Room along Bukit Timah,Central Region,Bukit Timah,Private room,81,90,18,2014-12-26,0.28,1,365,227796: Sujatha
2,COZICOMFORT,North Region,Woodlands,Private room,69,6,20,2015-10-01,0.20,2,365,266763: Francesca
3,Ensuite Room (Room 1 & 2) near EXPO,East Region,Tampines,Private room,206,1,14,2019-08-11,0.15,9,353,367042: Belinda
4,B&B Room 1 near Airport & EXPO,East Region,Tampines,Private room,94,1,22,2019-07-28,0.22,9,355,367042: Belinda


In [4]:
by_nbhd_group = airbnb.neighbourhood_group.value_counts()
by_nbhd = pd.value_counts(airbnb.neighbourhood)
by_room_type = pd.value_counts(airbnb.room_type)

print('Number of neighborhood groups: ' + str(by_nbhd_group.shape[0]))
print('Number of neighborhoods: ' + str(by_nbhd.shape[0]))
print('Number of room types: ' + str(by_room_type.shape[0]))

Number of neighborhood groups: 5
Number of neighborhoods: 43
Number of room types: 3


In [5]:
test = airbnb.groupby('neighbourhood_group')
test['price'].mean()

neighbourhood_group
Central Region       176.629418
East Region          142.503937
North Region         110.779412
North-East Region     99.838150
West Region          175.974074
Name: price, dtype: float64

In [6]:
group_dict = {'Central Region' : 0, 'East Region' : 1, 'North Region': 2, 'North-East Region': 3, 'West Region': 4 }
airbnbMapped = airbnb.copy()
airbnbMapped.neighbourhood_group = airbnbMapped.neighbourhood_group.map(group_dict)

central_region = airbnb[airbnbMapped.neighbourhood_group == 0]
east_region = airbnb[airbnbMapped.neighbourhood_group == 1]
north_region = airbnb[airbnbMapped.neighbourhood_group == 2]
north_east_region = airbnb[airbnbMapped.neighbourhood_group == 3]
west_region = airbnb[airbnbMapped.neighbourhood_group == 4]

In [18]:
Q1, Q3 = central_region.price.quantile([0.25, 0.75])
central_region_no = central_region[ (central_region.price >= Q1-1.5 * (Q3-Q1)) & (central_region.price <= Q3+1.5 * (Q3-Q1))]

Q1, Q3 = east_region.price.quantile([0.25, 0.75])
east_region_no = east_region[ (east_region.price >= Q1-1.5 * (Q3-Q1)) & (east_region.price <= Q3+1.5 * (Q3-Q1))]

Q1, Q3 = north_region.price.quantile([0.25, 0.75])
north_region_no = north_region[ (north_region.price >= Q1-1.5 * (Q3-Q1)) & (north_region.price <= Q3+1.5 * (Q3-Q1))]

Q1, Q3 = east_region.price.quantile([0.25, 0.75])
north_east_region_no = north_east_region[ (north_east_region.price >= Q1-1.5 * (Q3-Q1)) & (north_east_region.price <= Q3+1.5 * (Q3-Q1))]

Q1, Q3 = west_region.price.quantile([0.25, 0.75])
west_region_no = west_region[ (west_region.price >= Q1-1.5 * (Q3-Q1)) & (west_region.price <= Q3+1.5 * (Q3-Q1))]

In [24]:
print('Central Region Mean (w/ outliers): ' + str(central_region.price.mean()))
print('Central Region Mean (no outliers): ' + str(central_region_no.price.mean()))
print()
print('East Region Mean (w/ outliers): ' + str(east_region.price.mean()))
print('East Region Mean (no outliers): ' + str(east_region_no.price.mean()))
print()
print('North Region Mean (w/ outliers): ' + str(north_region.price.mean()))
print('North Region Mean (no outliers): ' + str(north_region_no.price.mean()))
print()
print('North-East Region Mean (w/ outliers): ' + str(north_east_region.price.mean()))
print('North-East Region Mean (no outliers): ' + str(north_east_region_no.price.mean()))
print()
print('West Region Mean (w/ outliers): ' + str(west_region.price.mean()))
print('West Region Mean (no outliers): ' + str(west_region_no.price.mean()))

Central Region Mean (w/ outliers): 176.62941829132984
Central Region Mean (no outliers): 143.45387823185987

East Region Mean (w/ outliers): 142.50393700787401
East Region Mean (no outliers): 101.97933884297521

North Region Mean (w/ outliers): 110.77941176470588
North Region Mean (no outliers): 84.1436170212766

North-East Region Mean (w/ outliers): 99.83815028901734
North-East Region Mean (no outliers): 76.53191489361703

West Region Mean (w/ outliers): 175.97407407407408
West Region Mean (no outliers): 84.36157024793388
